# 作業 : (Kaggle)鐵達尼生存預測
***
https://www.kaggle.com/c/titanic

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [9]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = 'data/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head(20)

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [10]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
df['Age'] = df['Age'].fillna('None')
mean_df = df.groupby(['Age'])['Fare'].mean().reset_index()
mode_df = df.groupby(['Age'])['Fare'].apply(lambda x: x.mode()[0]).reset_index()
median_df = df.groupby(['Age'])['Fare'].median().reset_index()
max_df = df.groupby(['Age'])['Fare'].max().reset_index()
temp = pd.merge(mean_df, mode_df, how='left', on=['Age'])
temp = pd.merge(temp, median_df, how='left', on=['Age'])
temp = pd.merge(temp, max_df, how='left', on=['Age'])
temp.columns = ['Age', 'Age_Fare_Mean', 'Age_Fare_Mode', 'Age_Fare_Median', 'Age_Fare_Max']
temp.head()
df = pd.merge(df, temp, how='left', on=['Age'])
df = df.drop(['Age'] , axis=1)
df.head()

,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_Fare_Mean,Age_Fare_Mode,Age_Fare_Median,Age_Fare_Max
0,3,"Braund, Mr. Owen Harris",male,1,0,A/5 21171,7.2500,NaN,S,25.504781,7.2500,7.89580,151.5500
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,1,0,PC 17599,71.2833,C85,C,62.751509,0.0000,31.38750,227.5250
2,3,"Heikkinen, Miss. Laina",female,0,0,STON/O2. 3101282,7.9250,NaN,S,19.086806,7.8542,9.58125,78.8500
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,1,0,113803,53.1000,C123,S,89.312500,8.0500,26.14375,512.3292
4,3,"Allen, Mr. William Henry",male,0,0,373450,8.0500,NaN,S,89.312500,8.0500,26.14375,512.3292


In [11]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

8 Numeric Features : ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age_Fare_Mean', 'Age_Fare_Mode', 'Age_Fare_Median', 'Age_Fare_Max']



,Pclass,SibSp,Parch,Fare,Age_Fare_Mean,Age_Fare_Mode,Age_Fare_Median,Age_Fare_Max
0,3,1,0,7.2500,25.504781,7.2500,7.89580,151.5500
1,1,1,0,71.2833,62.751509,0.0000,31.38750,227.5250
2,3,0,0,7.9250,19.086806,7.8542,9.58125,78.8500
3,1,1,0,53.1000,89.312500,8.0500,26.14375,512.3292
4,3,0,0,8.0500,89.312500,8.0500,26.14375,512.3292


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [13]:
# 原始特徵 + 邏輯斯迴歸
# 沒有這四個新特徵的 dataframe 稱為 df_minus
df_minus = df.drop(['Age_Fare_Mean', 'Age_Fare_Mode', 'Age_Fare_Median', 'Age_Fare_Max'] , axis=1)

train_X = MMEncoder.fit_transform(df_minus)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6847622178554621

In [15]:
# 新特徵 + 邏輯斯迴歸
train_X = MMEncoder.fit_transform(df)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6836322743526372